In [1]:
import json
from pprint import pprint as pp
import subprocess
from tqdm.notebook import tqdm
import numpy as np
import os
import re

In [2]:
path = r"C:\Users\Konrad Grudzinski\OneDrive - University of Glasgow\Computing\4th Year\Individual Project\Source\covid19\sorted_samples.json"

In [3]:
with open(path, "r") as f:
    data = json.load(f)

In [4]:
vaf_dir = "D:\\ncbi_dataset\\ncbi_dataset\\data\\vafs\\"
vaf_dir_wsl = "/mnt/d/ncbi_dataset/ncbi_dataset/data/vafs/"

vcf_dir = "D:\\ncbi_dataset\\ncbi_dataset\\data\\vcfs\\"
vcf_dir_wsl = "/mnt/d/ncbi_dataset/ncbi_dataset/data/vcfs/"

In [5]:
samples_path = "D:\\ncbi_dataset\\ncbi_dataset\\data\\split_genomics\\"
samples_path_wsl = "/mnt/d/ncbi_dataset/ncbi_dataset/data/split_genomics/"

In [6]:
def extract_vafs_and_depth(file: str) -> (list[float], int):
    vafs = []
    depths = []
    with open(file, "r") as f:
        line = f.readline()
        while line:
            columns = line.split("\t")
            if len(columns) == 10:
                vals = columns[9].split(":")
                cols = columns[8].split(":")
                if len(cols) == 4 and cols[3] == "VAF":
                    vafs.extend(list(map(lambda x:float(x), vals[3].split(","))))
                cols = columns[7].split(";")
                for c in cols:
                    if c.startswith("DP") and not c.startswith("DP4"):
                        depth = int(c.split("=")[1])
                        depths.append(depth)
                        break
            line = f.readline()
    return vafs, int(np.rint(np.mean(depths)))

In [7]:
def combine_fastas(output_path: str, input_samples: list[str]):
    with open(output_path, "w") as outF:
        for sample in input_samples:
            with open(samples_path + sample + ".fa", "r") as f:
                outF.write(f.read()+"\n")

In [12]:
def create_vcf_command(infile : str, outfile : str, counts : int, name : str):
    #commands = ["echo 'konny-ponny' | sudo --stdin -- ./createVCF.sh {} {}".format(infile, outfile)]
    #command = "eval $(tail ~/.bashrc -n 1) && {}\n".format(" && ".join(commands))
    threads = 6
    commands = []
    if name+".fa" in os.listdir("D:\\ncbi_dataset\\ncbi_dataset\\data\\aligned_samples"):
        aligned_file = f"/mnt/d/ncbi_dataset/ncbi_dataset/data/aligned_samples/{name}.fa"
    else:
        commands.append("echo \'Aligning....\'")
        if counts > 1000:
            commands.append(f"mafft --6merpair --thread {threads} --keeplength --addfragments {infile} NC_045512.2.fa | tee temp_aligned.fa > /dev/null")
        else:
            commands.append(f"mafft --thread {threads} --auto --add {infile} NC_045512.2.fa | tee temp_aligned.fa > /dev/null")
        commands.append("echo \'Copying aligned fasta....\'")
        commands.append(f"cp temp_aligned.fa /mnt/d/ncbi_dataset/ncbi_dataset/data/aligned_samples/{name}.fa")
        aligned_file = "temp_aligned.fa"
    commands.append("echo \'Creating SAM....\'")
    if counts < 100:
        commands.append(f"\'/mnt/c/Users/Konrad Grudzinski/OneDrive - University of Glasgow/Computing/4th Year/Individual Project/Source/hisat2-2.2.1/hisat2\' -x NC_045512.2.fa -f {aligned_file} -S temp_all.sam --threads {threads} -k 50 --score-min L,0,-2")
    else:
        commands.append(f"\'/mnt/c/Users/Konrad Grudzinski/OneDrive - University of Glasgow/Computing/4th Year/Individual Project/Source/hisat2-2.2.1/hisat2\' -x NC_045512.2.fa -f {aligned_file} -S temp_all.sam --threads {threads}")
    commands.append("echo \'Sorting....\'")
    commands.append(f"samtools sort --verbosity 10 --threads {threads} temp_all.sam -o temp_all-sorted.bam")
    commands.append("echo \'Pileup....\'")
    commands.append(f"bcftools mpileup -a FORMAT/AD,INFO/AD -d 120000 --threads {threads} -f NC_045512.2.fa temp_all-sorted.bam | tee temp_pileup > /dev/null")
    commands.append("echo \'Calling....\'")
    commands.append("bcftools call --ploidy 1 -mA temp_pileup | tee temp_called > /dev/null")
    #commands.append("bcftools filter -s LowQual -e \'%QUAL<20 || DP>100\' temp_called | tee {VCF} > /dev/null".format(VCF=outfile+".vcf"))
    commands.append("echo \'Annotating....\'")
    commands.append(f"bcftools +fill-tags temp_called -Ov -o {outfile+'.vcf'} -- -t AF,FORMAT/VAF")
    
    returncode = os.system('start /wait cmd /c wsl -- eval "{}"\n'.format(" && ".join(commands).replace("bcftools", "\'/mnt/c/Users/Konrad Grudzinski/OneDrive - University of Glasgow/Computing/4th Year/Individual Project/Source/bcftools-1.14/bcftools\'")))
    if returncode != 0:
        print(returncode)

In [13]:
for country, info in tqdm(list(data.items())):
    print(country)
    country = country.replace(" ", "-")
    for month, month_samples in tqdm(list(info["months"].items())):
        print(month, month_samples["counts"], sep="\t")
        if month_samples["counts"] < 2:
            continue
        name = "m_{}_{}_{}".format(country, month, month_samples["counts"])
        temp_combined = name+".temp.fa"
        if name+".vcf" not in os.listdir(vcf_dir): # check if vcf has not been created already
            if temp_combined not in os.listdir():
                combine_fastas(temp_combined, month_samples["samples"])
            vcf_path_wsl = vcf_dir_wsl + name
            create_vcf_command(temp_combined, vcf_path_wsl, month_samples["counts"], name)
        r = re.compile("m_{}_{}_[0-9]+\.vaf".format(country, month))
        if len(list(filter(r.match, os.listdir(vaf_dir)))) == 0: # check if vaf has not been created already
            vcf_path = vcf_dir + name + ".vcf"
            vafs, avg_depth = extract_vafs_and_depth(vcf_path)
            name = "m_{}_{}_{}".format(country, month, avg_depth)
            vaf_path = vaf_dir + name + ".vaf"
            with open(vaf_path, "w") as f:
                f.write("\n".join(map(lambda x:str(x), vafs)))
        try:
            os.remove(temp_combined)
        except:
            pass
    print("----------------------------------------------------")

  0%|          | 0/87 [00:00<?, ?it/s]

Japan


  0%|          | 0/19 [00:00<?, ?it/s]

2020-4	98
2020-3	41
2020-5	29
2020-6	31
2020-7	55
2020-8	46
2020-11	89
2020-10	41
2020-9	38
2020-12	152
2020-2	73
2021-2	32
2021-1	18
2021-3	9
2021-6	109
2021-4	86
2021-5	39
2021-7	81
2020-1	1
----------------------------------------------------
United Kingdom


  0%|          | 0/23 [00:00<?, ?it/s]

...

2020-8	1
2020-9	1
2020-7	1
2021-8	4
2021-7	16
2021-9	1
2021-2	5
2021-1	1
2021-6	2
----------------------------------------------------
Mongolia


  0%|          | 0/2 [00:00<?, ?it/s]

2021-7	18
2021-8	40
----------------------------------------------------
Slovakia


  0%|          | 0/12 [00:00<?, ?it/s]

2021-4	763
2021-3	1088
2021-2	64
2021-5	857
2021-6	562
2021-7	544
2021-8	832
2021-10	1575
2021-9	1413
2021-1	34
2021-11	1812
2021-12	4
----------------------------------------------------
